In [1]:
# Importar librerías necesarias
import pandas as pd

In [2]:
# Definir la ruta al archivo CSV si no está definida
file_path_precio_m2 = '/Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_raw.csv'

# Cargar el archivo malaga_precio_m2.csv usando la variable ya definida
try:
    precio_m2_df = pd.read_csv(file_path_precio_m2, encoding='utf-8')
except UnicodeDecodeError:
    precio_m2_df = pd.read_csv(file_path_precio_m2, encoding='latin1')

# Buscar el nombre de la columna con "Mes" y "A" para renombrar correctamente
historical_max_date_col = [col for col in precio_m2_df.columns if "Mes" in col and "A" in col]
rename_dict = {
    'Localización': 'neighbourhood',
    'Precio m2 jun 2025': 'price_per_m2',
    'Variación mensual': 'monthly_variation',
    'Variación trimestral': 'quarterly_variation',
    'Variación anual': 'annual_variation',
    'Máximo histórico': 'historical_max',
    'Variación máximo': 'historical_max_variation'
}
if historical_max_date_col:
    rename_dict[historical_max_date_col[0]] = 'historical_max_date'

precio_m2_df.rename(columns=rename_dict, inplace=True)

# Definir las columnas a limpiar
columns_to_clean = ['monthly_variation', 'quarterly_variation', 'annual_variation', 'historical_max_variation']

# Limpiar nombres de zonas para que coincidan con malaga_clean.csv
if 'neighbourhood' in precio_m2_df.columns:
    precio_m2_df['neighbourhood'] = precio_m2_df['neighbourhood'].astype(str).str.strip()

# Mostrar el DataFrame limpio
print("DataFrame limpio:")
display(precio_m2_df.head())

DataFrame limpio:


,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_variation
0,Málaga,3.459 €/m2,"+ 0,9 %","+ 4,2 %","+ 14,7 %",3.459 €/m2 jun 2025,"0,0 %"
1,Bailén - Miraflores,2.845 €/m2,"- 0,1 %","+ 3,7 %","+ 29,8 %",2.849 €/m2 mayo 2025,"- 0,1 %"
2,Carretera de Cádiz,3.519 €/m2,"+ 0,8 %","+ 1,8 %","+ 11,6 %",3.519 €/m2 jun 2025,"0,0 %"
3,Centro,4.003 €/m2,"- 0,4 %","+ 0,4 %","+ 6,1 %",4.035 €/m2 abr 2025,"- 0,8 %"
4,Churriana,2.860 €/m2,"+ 2,9 %","+ 6,9 %","+ 25,7 %",2.860 €/m2 jun 2025,"0,0 %"


In [3]:
# Dividir la columna 'historical_max' en valor y fecha usando '€/m2'
precio_m2_df[['historical_max', 'historical_max_date']] = (
    precio_m2_df['historical_max']
    .str.split('€/m2', n=1, expand=True)
)

# Limpiar los resultados de la división
precio_m2_df['historical_max'] = precio_m2_df['historical_max'].str.replace('.', '', regex=False).str.strip()
precio_m2_df['historical_max_date'] = precio_m2_df['historical_max_date'].str.strip()

# Reordenar las columnas si es necesario
precio_m2_df = precio_m2_df[['neighbourhood', 'price_per_m2', 'monthly_variation', 'quarterly_variation',
                             'annual_variation', 'historical_max', 'historical_max_date',
                             'historical_max_variation']]

# Mostrar todas las filas del DataFrame limpio
print("DataFrame limpio:")
display(precio_m2_df.head())

DataFrame limpio:


,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Málaga,3.459 €/m2,"+ 0,9 %","+ 4,2 %","+ 14,7 %",3459,jun 2025,"0,0 %"
1,Bailén - Miraflores,2.845 €/m2,"- 0,1 %","+ 3,7 %","+ 29,8 %",2849,mayo 2025,"- 0,1 %"
2,Carretera de Cádiz,3.519 €/m2,"+ 0,8 %","+ 1,8 %","+ 11,6 %",3519,jun 2025,"0,0 %"
3,Centro,4.003 €/m2,"- 0,4 %","+ 0,4 %","+ 6,1 %",4035,abr 2025,"- 0,8 %"
4,Churriana,2.860 €/m2,"+ 2,9 %","+ 6,9 %","+ 25,7 %",2860,jun 2025,"0,0 %"


In [4]:
# Insertar manual para completar el DataFrame con los barrios que faltan, 'Campanillas' y 'Palma-Palmilla'

# Nuevas filas para Campanillas y Palma-Palmilla
new_data = pd.DataFrame([
    {
        "neighbourhood": "Campanillas",
        "price_per_m2": "2165 €/m2",
        "monthly_variation": "+4.8 %",
        "quarterly_variation": "+5.6 %",
        "annual_variation": "+4.8 %",
        "historical_max": "2268",
        "historical_max_date": "dic 2024",
        "historical_max_variation": "-4.5 %"
    },
    {
        "neighbourhood": "Palma-Palmilla",
        "price_per_m2": "2851 €/m2",
        "monthly_variation": "0.0",
        "quarterly_variation": "0.0",
        "annual_variation": "0.0",
        "historical_max": "2851",
        "historical_max_date": "mayo 2025",
        "historical_max_variation": "0.0 %"
    }
])

# Reemplazar 'n.d.' por 0.0 en la columna 'monthly_variation'
new_data['monthly_variation'] = (
    new_data['monthly_variation']
    .astype(str)  # Convertir a cadena para manipulación
    .str.strip()  # Eliminar espacios adicionales
    .str.replace('n.d.', '0.0', regex=False)  # Reemplazar 'n.d.' por '0.0'
)

# Combinar con el DataFrame original
precio_m2_df = pd.concat([precio_m2_df, new_data], ignore_index=True)

# Reemplazar 'n.d.' en el DataFrame principal también, si es necesario
precio_m2_df['monthly_variation'] = (
    precio_m2_df['monthly_variation']
    .astype(str)  # Convertir a cadena para manipulación
    .str.strip()  # Eliminar espacios adicionales
    .str.replace('n.d.', '0.0', regex=False)  # Reemplazar 'n.d.' por '0.0'
)

# Mostrar el resultado
precio_m2_df.head(15)

,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Málaga,3.459 €/m2,"+ 0,9 %","+ 4,2 %","+ 14,7 %",3459,jun 2025,"0,0 %"
1,Bailén - Miraflores,2.845 €/m2,"- 0,1 %","+ 3,7 %","+ 29,8 %",2849,mayo 2025,"- 0,1 %"
2,Carretera de Cádiz,3.519 €/m2,"+ 0,8 %","+ 1,8 %","+ 11,6 %",3519,jun 2025,"0,0 %"
3,Centro,4.003 €/m2,"- 0,4 %","+ 0,4 %","+ 6,1 %",4035,abr 2025,"- 0,8 %"
4,Churriana,2.860 €/m2,"+ 2,9 %","+ 6,9 %","+ 25,7 %",2860,jun 2025,"0,0 %"
5,Ciudad Jardín,2.296 €/m2,0.0,"+ 9,1 %","+ 18,2 %",2296,jun 2025,"0,0 %"
6,Cruz de Humilladero,3.045 €/m2,"+ 2,3 %","+ 6,5 %","+ 20,5 %",3045,jun 2025,"0,0 %"
7,Este,4.466 €/m2,"+ 0,3 %","+ 4,1 %","+ 10,8 %",4466,jun 2025,"0,0 %"
8,Puerto de la Torre,2.540 €/m2,"+ 2,2 %","+ 0,4 %","+ 6,6 %",2547,nov 2024,"- 0,3 %"
9,Teatinos,3.554 €/m2,"+ 0,3 %","+ 5,1 %","+ 11,7 %",3576,dic 2024,"- 0,6 %"


In [5]:
# Remover '€/m2' de las columnas que contienen precios
precio_m2_df['price_per_m2'] = (
    precio_m2_df['price_per_m2']
    .astype(str)
    .str.replace('€/m2', '', regex=False)  # Eliminar '€/m2'
    .str.strip()  # Eliminar espacios innecesarios
)

# Remover '.' de las columnas de precios 'historical_max' y 'price_per_m2'
precio_m2_df['historical_max'] = precio_m2_df['historical_max'].str.replace('.', '', regex=False).astype(float)
precio_m2_df['price_per_m2'] = precio_m2_df['price_per_m2'].str.replace('.', '', regex=False).astype(float)

# Remover tildes de la columna 'neighbourhood'
precio_m2_df['neighbourhood'] = (
    precio_m2_df['neighbourhood']
    .str.normalize('NFD')  # Normalizar caracteres
    .str.encode('ascii', errors='ignore')  # Eliminar caracteres especiales
    .str.decode('utf-8')  # Volver a convertir a string
    .str.strip()  # Eliminar espacios innecesarios
)

In [6]:
precio_m2_df.info()  # Mostrar información del DataFrame limpio

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   neighbourhood             12 non-null     object 
 1   price_per_m2              12 non-null     float64
 2   monthly_variation         12 non-null     object 
 3   quarterly_variation       12 non-null     object 
 4   annual_variation          12 non-null     object 
 5   historical_max            12 non-null     float64
 6   historical_max_date       12 non-null     object 
 7   historical_max_variation  12 non-null     object 
dtypes: float64(2), object(6)
memory usage: 900.0+ bytes


In [7]:
# Corregir tipos de datos para columnas numéricas

# Redondear columnas floats a 2 decimales
float_columns = ['price_per_m2', 'monthly_variation', 'quarterly_variation', 'annual_variation', 'historical_max', 'historical_max_variation']
for col in float_columns:
    if col in precio_m2_df.columns:
        # Asegurarse de que los valores sean numéricos antes de redondear
        precio_m2_df[col] = (
            precio_m2_df[col]
            .astype(str)  # Convertir a string para limpieza
            .str.replace(',', '.', regex=False)  # Reemplazar comas por puntos
            .str.replace('%', '', regex=False)  # Eliminar el símbolo de porcentaje
            .str.replace('+', '', regex=False)  # Eliminar el símbolo de positivo
            .str.replace('−', '-', regex=False)  # Asegurar que los signos negativos Unicode sean correctos
            .str.replace('- ', '-', regex=False)  # Eliminar espacios entre el signo negativo y el número
            .str.strip()  # Eliminar espacios innecesarios
        )
        # Convertir a numérico y redondear a 2 decimales
        precio_m2_df[col] = pd.to_numeric(precio_m2_df[col], errors='coerce').round(2)

# Verificar los cambios realizados
print("DataFrame después de las correcciones:")
precio_m2_df.info()

# Mostrar las primeras filas para verificar los valores negativos
print("\nPrimeras filas del DataFrame:")
display(precio_m2_df.head(15))

DataFrame después de las correcciones:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   neighbourhood             12 non-null     object 
 1   price_per_m2              12 non-null     float64
 2   monthly_variation         12 non-null     float64
 3   quarterly_variation       12 non-null     float64
 4   annual_variation          12 non-null     float64
 5   historical_max            12 non-null     float64
 6   historical_max_date       12 non-null     object 
 7   historical_max_variation  12 non-null     float64
dtypes: float64(6), object(2)
memory usage: 900.0+ bytes

Primeras filas del DataFrame:


,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Malaga,3459.0,0.9,4.2,14.7,3459.0,jun 2025,0.0
1,Bailen - Miraflores,2845.0,-0.1,3.7,29.8,2849.0,mayo 2025,-0.1
2,Carretera de Cadiz,3519.0,0.8,1.8,11.6,3519.0,jun 2025,0.0
3,Centro,4003.0,-0.4,0.4,6.1,4035.0,abr 2025,-0.8
4,Churriana,2860.0,2.9,6.9,25.7,2860.0,jun 2025,0.0
5,Ciudad Jardin,2296.0,0.0,9.1,18.2,2296.0,jun 2025,0.0
6,Cruz de Humilladero,3045.0,2.3,6.5,20.5,3045.0,jun 2025,0.0
7,Este,4466.0,0.3,4.1,10.8,4466.0,jun 2025,0.0
8,Puerto de la Torre,2540.0,2.2,0.4,6.6,2547.0,nov 2024,-0.3
9,Teatinos,3554.0,0.3,5.1,11.7,3576.0,dic 2024,-0.6


In [8]:
# Imprimir valores unicos en 'neighbourhood'
print("Valores únicos en 'neighbourhood':")
print(precio_m2_df['neighbourhood'].unique())

Valores únicos en 'neighbourhood':
['Malaga' 'Bailen - Miraflores' 'Carretera de Cadiz' 'Centro' 'Churriana'
 'Ciudad Jardin' 'Cruz de Humilladero' 'Este' 'Puerto de la Torre'
 'Teatinos' 'Campanillas' 'Palma-Palmilla']


In [9]:
# Función para estandarizar los valores de 'neighbourhood'
def standardize_neighbourhood_values(df):    
    if 'neighbourhood' in df.columns:
        df['neighbourhood'] = (
            df['neighbourhood']
            .astype(str)
            .str.strip()
            .str.replace(r'\s*-\s*', '-', regex=True)  # Remove spaces around '-'
        )
    return df

# Aplicar la función de estandarización a los valores de 'neighbourhood'
precio_m2_df = standardize_neighbourhood_values(precio_m2_df)

# Estandarizar los valores de 'neighbourhood' para que coincidan con los de malaga_listings_clean.csv
# Unificar valores de 'neighbourhood' para mayor consistencia
replacements = {
    'Teatinos': 'Teatinos-Universidad',
    'Cruz de Humilladero': 'Cruz De Humilladero'
}
precio_m2_df['neighbourhood'] = precio_m2_df['neighbourhood'].replace(replacements, regex=False)


# Verificar los cambios realizados
print("Valores únicos en 'neighbourhood' después de la estandarización:")
print(precio_m2_df['neighbourhood'].unique())

Valores únicos en 'neighbourhood' después de la estandarización:
['Malaga' 'Bailen-Miraflores' 'Carretera de Cadiz' 'Centro' 'Churriana'
 'Ciudad Jardin' 'Cruz De Humilladero' 'Este' 'Puerto de la Torre'
 'Teatinos-Universidad' 'Campanillas' 'Palma-Palmilla']


In [10]:
# Mostrar el DataFrame final
print("DataFrame final:")
display(precio_m2_df.head(15))

DataFrame final:


,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Malaga,3459.0,0.9,4.2,14.7,3459.0,jun 2025,0.0
1,Bailen-Miraflores,2845.0,-0.1,3.7,29.8,2849.0,mayo 2025,-0.1
2,Carretera de Cadiz,3519.0,0.8,1.8,11.6,3519.0,jun 2025,0.0
3,Centro,4003.0,-0.4,0.4,6.1,4035.0,abr 2025,-0.8
4,Churriana,2860.0,2.9,6.9,25.7,2860.0,jun 2025,0.0
5,Ciudad Jardin,2296.0,0.0,9.1,18.2,2296.0,jun 2025,0.0
6,Cruz De Humilladero,3045.0,2.3,6.5,20.5,3045.0,jun 2025,0.0
7,Este,4466.0,0.3,4.1,10.8,4466.0,jun 2025,0.0
8,Puerto de la Torre,2540.0,2.2,0.4,6.6,2547.0,nov 2024,-0.3
9,Teatinos-Universidad,3554.0,0.3,5.1,11.7,3576.0,dic 2024,-0.6


In [11]:
# Guardar el archivo limpio
output_path_precio_m2 = '/Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_clean.csv'
precio_m2_df.to_csv(output_path_precio_m2, index=False)

print(f"Archivo limpio guardado en: {output_path_precio_m2}")

Archivo limpio guardado en: /Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_clean.csv
